In [1]:
import sys
import os

# Add the parent directory of the 'src' directory to the Python path
from sglang.srt.server import Runtime as SGLangServer
from sglang.srt.managers.io_struct import SchedulingMetricsReqInput, MigrationReq
import requests
import concurrent.futures
import unittest

/mnt/ssd1/alm-os/sg_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-18 01:50:49,880	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
server_url = 'http://localhost:30000'

r = requests.get(f'{server_url}/get_model_info')
r

<Response [200]>

In [3]:
add_req_url = f'{server_url}/add_request'
r = requests.post(add_req_url, json={ 
    "text": "Say this is a warmup request.",
    "sampling_params": {
        "temperature": 0,
        "max_new_tokens": 16,
    },
})
r

<Response [200]>

In [4]:
metrics_url = f"{server_url}/scheduling_metrics"
metrics_results1 = requests.post(metrics_url, json={"prompt": "sample prompt"})
metrics_data1 = metrics_results1.json()
metrics_data1

{'waiting_queue_len': 0,
 'running_req_len': 0,
 'prefix_match_len': 0,
 'token_kv_available_size': 237245,
 'evicatable_size': 0,
 'tree_cache_metrics_hit': 0,
 'tree_cache_metrics_total': 0,
 'input_len': 3,
 'total_radix_cache_processing_time': 0.017175674438476562,
 'queue_processing_time': 0.0006718635559082031,
 'inner_router_time': 0.017209768295288086,
 'matching_overhead': 0.011205673217773438,
 'tokenization_time': 0.00027823448181152344,
 'routing_time': 0.0002167224884033203,
 'return_time': 3.5762786865234375e-06,
 'request_processing_time': 8.249282836914062e-05,
 'total_internal_request_time': 0.019067049026489258}

In [5]:
import zmq
import zmq.asyncio

In [6]:
context = zmq.asyncio.Context()
recv_from_rounter = context.socket(zmq.PULL)
recv_from_rounter.bind(f"tcp://127.0.0.1:23334")

<SocketContext(bind='tcp://127.0.0.1:23334')>

In [7]:
migrate_url = f"{server_url}/migrate_control"
metrics_results1 = requests.post(migrate_url, params={"migration_target_url": "tcp://127.0.0.1:23334"})
mreq = await recv_from_rounter.recv_pyobj()

In [8]:
mreq.requets

[rid(n=0e13fc247dd14a8db5692b36f5cedae4, input_ids=[1, 15753, 456, 349, 264, 6100, 715, 2159, 28723], ,
 rid(n=72c77158221b45bfbd54ea08d36c23b6, input_ids=[1, 15753, 456, 349, 264, 6100, 715, 2159, 28723], ,
 rid(n=b513d4d97fce4e799d1af84f5b3f9999, input_ids=[1, 15753, 456, 349, 264, 6100, 715, 2159, 28723], ]